In [1]:
import json
import os
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

SCORING_MODEL = os.environ["SCORING_MODEL"]

# Specify JSON file to load (from .artifacts directory)
# Format: City_CheckinDate_CheckoutDate.json
ARTIFACT_FILE = "Иркутск_2026-03-01_2026-03-04.json"  # Change this to your file

# Load data from JSON
artifacts_dir = Path(".artifacts")
filepath = artifacts_dir / ARTIFACT_FILE

if not filepath.exists():
    raise FileNotFoundError(f"File not found: {filepath}")

with open(filepath, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract search parameters
search = data["search"]
CITY = search["city"]
REGION_ID = search["region_id"]
CHECKIN_DATE = search["checkin"]
CHECKOUT_DATE = search["checkout"]
GUESTS = search["guests"]
MIN_PRICE = search["min_price"]
MAX_PRICE = search["max_price"]
CURRENCY = search["currency"]
LANGUAGE = search["language"]
RESIDENCY = search["residency"]

# Extract hotels data
combined = data["hotels"]  # HotelFull[]
stats = data["stats"]

# Reconstruct reviews_map from combined data
reviews_map = {}
for hotel in combined:
    hid = hotel.get("hid")
    reviews = hotel.get("reviews")
    if hid and reviews:
        reviews_map[hid] = reviews

print(f"✅ Loaded data from {ARTIFACT_FILE}")
print(f"   Search: {CITY}, {CHECKIN_DATE} → {CHECKOUT_DATE}")
print(f"   Hotels: {len(combined)}")
print(f"   Stats: {stats}")

# Load reference CSV if exists
ref_filename = f"REF_{Path(ARTIFACT_FILE).stem}.csv"
ref_filepath = artifacts_dir / ref_filename

reference_scores = {}
if ref_filepath.exists():
    ref_df = pd.read_csv(ref_filepath)
    # Extract hid, name, evf_score into dict: {hid: {"name": name, "evf_score": score}}
    for _, row in ref_df.iterrows():
        hid = row.get("hid")
        name = row.get("name")
        evf_score = row.get("evf_score")
        if hid and evf_score is not None:
            reference_scores[hid] = {"name": name, "evf_score": evf_score}
    print(f"✅ Loaded reference scores from {ref_filename}")
    print(f"   Reference hotels: {len(reference_scores)}")
else:
    print(f"ℹ️  No reference file found: {ref_filename}")

# User preferences for scoring (can be modified)
USER_PREFERENCES = "Обязательно две комнаты и две кровати. Хорошие отзывы. Чистота"

✅ Loaded data from Иркутск_2026-03-01_2026-03-04.json
   Search: Иркутск, 2026-03-01 → 2026-03-04
   Hotels: 335
   Stats: {'total_hotels': 335, 'total_available': 485, 'total_after_filter': 335}
✅ Loaded reference scores from REF_Иркутск_2026-03-01_2026-03-04.csv
   Reference hotels: 120


In [2]:
# Placeholder cell

In [3]:
import json

from services import estimate_tokens, prepare_hotel_for_llm, presort_hotels

# Parameters for review sampling
MAX_REVIEWS_PER_HOTEL = 30
REVIEW_TEXT_MAX_LENGTH = 512

# Estimate tokens before presort
hotels_for_llm_all = [
    prepare_hotel_for_llm(h, MIN_PRICE, MAX_PRICE, MAX_REVIEWS_PER_HOTEL, REVIEW_TEXT_MAX_LENGTH)
    for h in combined
]
tokens_before = estimate_tokens(json.dumps(hotels_for_llm_all, ensure_ascii=False), SCORING_MODEL)

# Pre-sort by hotel kind tier and prescore, limit to top 100 for LLM scoring
PRESORT_LIMIT = 120
top_hotels = presort_hotels(combined, reviews_map, limit=PRESORT_LIMIT)

# Estimate tokens after presort
hotels_for_llm_top = [
    prepare_hotel_for_llm(h, MIN_PRICE, MAX_PRICE, MAX_REVIEWS_PER_HOTEL, REVIEW_TEXT_MAX_LENGTH)
    for h in top_hotels
]
tokens_after = estimate_tokens(json.dumps(hotels_for_llm_top, ensure_ascii=False), SCORING_MODEL)

print(f"[presort_done] {len(combined)} отелей → {len(top_hotels)} (лимит {PRESORT_LIMIT})")
print(f"  Токены: ~{tokens_before:,} → ~{tokens_after:,} (экономия {tokens_before - tokens_after:,})")

[presort_done] 335 отелей → 120 (лимит 120)
  Токены: ~339,951 → ~209,745 (экономия 130,206)


In [4]:
# Check if all reference hotels are in top_hotels
if reference_scores:
    top_hotel_hids = {h["hid"] for h in top_hotels}
    missing_hotels = []

    for hid, ref_data in reference_scores.items():
        if hid not in top_hotel_hids:
            missing_hotels.append({
                "hid": hid,
                "name": ref_data["name"],
                "evf_score": ref_data["evf_score"]
            })

    if missing_hotels:
        print(f"\n⚠️  {len(missing_hotels)} референсных отелей отсутствуют в top_hotels:")
        for hotel in missing_hotels:
            print(f"   - {hotel['name']} (hid: {hotel['hid']}, evf_score: {hotel['evf_score']})")
    else:
        print(f"\n✅ Все {len(reference_scores)} референсных отелей присутствуют в top_hotels")
    
    # Check reverse: hotels in top_hotels but not in reference
    ref_hids = set(reference_scores.keys())
    hotels_not_in_ref = []
    
    for hotel in top_hotels:
        hid = hotel.get("hid")
        if hid and hid not in ref_hids:
            hotels_not_in_ref.append(hotel)
    
    if hotels_not_in_ref:
        print(f"\n⚠️  {len(hotels_not_in_ref)} отелей из top_hotels отсутствуют в референсе:")
        print(f"{'='*120}\n")
        
        for hotel in hotels_not_in_ref:
            hotel_id = hotel.get("id", "")
            hid = hotel.get("hid", "")
            hotel_chain = hotel.get("hotel_chain", "No chain")
            kind = hotel.get("kind", "")
            name = hotel.get("name", "")
            metapolicy = hotel.get("metapolicy_struct", {})
            stars = hotel.get("stars", 0)
            serp_filters = hotel.get("serp_filters", [])
            facts = hotel.get("facts_summary", {})
            
            print(f"{hotel_id} {hid} {hotel_chain} {kind} {name} {metapolicy} {stars} {serp_filters} {facts}")
            print()
    else:
        print(f"\n✅ Все отели из top_hotels присутствуют в референсе")


✅ Все 120 референсных отелей присутствуют в top_hotels

✅ Все отели из top_hotels присутствуют в референсе


In [ ]:
import time

from services import finalize_scored_hotels, score_hotels

# Score hotels using single LLM request
print(f"[scoring_start] Scoring {len(top_hotels)} hotels...")
start_time = time.time()

scoring_result = await score_hotels(
    top_hotels,
    USER_PREFERENCES,
    guests=GUESTS,
    max_reviews=MAX_REVIEWS_PER_HOTEL,
    review_text_max_length=REVIEW_TEXT_MAX_LENGTH,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    currency=CURRENCY,
    top_count=10
)

elapsed = time.time() - start_time

if scoring_result["error"]:
    print(f"\n❌ ERROR: {scoring_result['error']}")
    scored_hotels = None
else:
    scoring_results = scoring_result["results"]
    print(f"[scoring_done] {len(scoring_results)} hotels scored — {elapsed:.1f}s")
    print(f"  Estimated tokens: ~{scoring_result['estimated_tokens']:,}")
    
    # Finalize scored hotels - merge scoring results with full hotel data
    scored_hotels = finalize_scored_hotels(combined, scoring_results)
    print(f"\n[finalize_done] {len(scored_hotels)} hotels with complete data")

In [9]:
from services import HotelScored
from utils import ostrovok_url

def display_top_hotels(
    scored_hotels: list[HotelScored],
    top_n: int = 10,
) -> pd.DataFrame:
    """Display top N scored hotels with details and Ostrovok links."""
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    data = []
    for i, hotel in enumerate(scored_hotels[:top_n], 1):
        hotel_id = hotel["id"]
        name = hotel["name"]
        hid = hotel["hid"]
        kind = hotel.get("kind", "")
        score = hotel["score"]
        reasons = hotel.get("top_reasons", [])
        penalties = hotel.get("score_penalties", [])
        selected_hash = hotel.get("selected_rate_hash")

        # Find selected rate by hash
        rates = hotel.get("rates", [])
        selected_rate = next((r for r in rates if r.get("match_hash") == selected_hash), None)

        # Get rate details
        if selected_rate:
            room_name = selected_rate.get("room_name", "")[:50]
            meal_data = selected_rate.get("meal_data", {})
            meal = meal_data.get("value", selected_rate.get("meal", ""))

            # Calculate prices from daily_prices
            daily_prices = selected_rate.get("daily_prices", [])
            if daily_prices:
                # Convert string prices to float and sum
                total_price = sum(float(p) for p in daily_prices)
                num_nights = len(daily_prices)
                avg_price_per_night = total_price / num_nights if num_nights > 0 else 0

                # Get currency
                pt = selected_rate.get("payment_options", {}).get("payment_types", [])
                currency = pt[0].get("show_currency_code", "") if pt else ""

                total_price_str = f"{total_price:.0f} {currency}"
                avg_price_str = f"{avg_price_per_night:.0f} {currency}"
            else:
                # Fallback to payment_types if daily_prices not available
                pt = selected_rate.get("payment_options", {}).get("payment_types", [])
                if pt:
                    total_price = float(pt[0].get("show_amount", 0))
                    currency = pt[0].get("show_currency_code", "")
                    total_price_str = f"{total_price:.0f} {currency}"
                    avg_price_str = f"{total_price:.0f} {currency}"
                else:
                    total_price_str = "N/A"
                    avg_price_str = "N/A"
        else:
            room_name = "N/A"
            meal = "N/A"
            total_price_str = "N/A"
            avg_price_str = "N/A"

        # Get reviews data
        reviews = hotel.get("reviews")
        avg_rating = reviews.get("avg_rating") if reviews else None
        detailed = reviews.get("detailed_averages", {}) if reviews else {}

        # Generate Ostrovok URL
        url = ostrovok_url(
            hotel_id=hotel_id,
            hid=hid,
            checkin=CHECKIN_DATE,
            checkout=CHECKOUT_DATE,
            guests=GUESTS,
            region_id=REGION_ID,
        )

        # Print detailed info
        print(f"{i}. {name} [{kind}]")
        print(f"   Score: {score}/100 | Rating: {avg_rating}/10" if avg_rating else f"   Score: {score}/100")
        print(f"   Room: {room_name}")
        print(f"   Total: {total_price_str} | Avg per night: {avg_price_str} | Meal: {meal}")
        if reasons:
            print(f"   + {'; '.join(reasons[:3])}")
        if penalties:
            print(f"   - {'; '.join(penalties[:5])}")
        print(f"   🔗 {url}")
        print()

        # Collect for DataFrame
        data.append({
            "name": name[:35],
            "kind": kind,
            "room": room_name[:30],
            "total": total_price_str,
            "avg/night": avg_price_str,
            "meal": meal,
            "score": score,
            "rating": avg_rating,
            "clean": detailed.get("cleanness"),
            "url": url,
        })

    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    selected = min(top_n, len(scored_hotels))
    print(f"Всего найдено {len(combined)} отелей на эти даты.")
    print(f"Подобраны лучшие {selected} по вашим критериям.")
    return df


pd.set_option("display.max_colwidth", 100)
display_top_hotels(scored_hotels, top_n=10)

,name,kind,room,total,avg/night,meal,score,rating,clean,url
1,Апартаменты MORE и SPA c сауной и в,Resort,Двухместные апартаменты с 2 ко,30220 RUB,10073 RUB,nomeal,98,10.0,10.0,https://ostrovok.ru/hotel/russia/moscow/mid13319787/more_view_spa_rasko_home_s_saunoy_apartments...
2,Квартира двухкомнатные Be Home на 2,Apartment,Апартаменты,11293 RUB,3764 RUB,nomeal,96,9.8,9.9,https://ostrovok.ru/hotel/russia/moscow/mid9992650/sibirskie_apartamentyi_na_25_oktyabrya/?dates...
3,Апартаменты Изумруд Возле Аэропорта,Apartment,Двухместные апартаменты Standa,11760 RUB,3920 RUB,nomeal,94,9.6,9.6,https://ostrovok.ru/hotel/russia/moscow/mid10002890/aeroport_kultukskaya_home_apartments/?dates=...
4,Квартира Двухкомнатная Студя в Цент,Apartment,Четырёхместные апартаменты (ди,11111 RUB,3704 RUB,nomeal,93,9.5,9.6,https://ostrovok.ru/hotel/russia/moscow/mid10095054/apartamenty_dvukhkomnatnaya_studya_v_tsentre...
5,Апартаменты Красных Мадьяр 14,Apartment,Апартаменты Superior с 2 комна,11525 RUB,3842 RUB,nomeal,92,9.5,9.6,https://ostrovok.ru/hotel/russia/moscow/mid9769032/krasnyih_madyar_14_apartments/?dates=01.03.20...
6,Апартаменты Колибри Премиум Класса,Apartment,Апартаменты с балконом и с кра,20799 RUB,6933 RUB,nomeal,90,9.2,9.2,https://ostrovok.ru/hotel/russia/moscow/mid10530830/kolibri_apartments_3/?dates=01.03.2026-04.03...
7,AZIMUT Отель Аэропорт Иркутск 4*,Hotel,Двухместный люкс (двуспальная,44860 RUB,14953 RUB,breakfast,89,9.5,10.0,https://ostrovok.ru/hotel/russia/moscow/mid13317260/azimut_aeroport_irkutsk_hotel/?dates=01.03.2...
8,Отель Иркутск,Hotel,Двухместный люкс Executive (дв,34800 RUB,11600 RUB,nomeal,85,8.0,8.0,https://ostrovok.ru/hotel/russia/moscow/mid7819551/irkutsk/?dates=01.03.2026-04.03.2026&guests=2...
9,Отель Baikal Forest,Hotel,Двухместный Suite Business (дв,38016 RUB,12672 RUB,nomeal,84,8.2,8.8,https://ostrovok.ru/hotel/russia/moscow/mid9825031/baikal_forest_hotel/?dates=01.03.2026-04.03.2...
10,Отель North Star,Hotel,Двухместный семейный люкс с кр,29400 RUB,9800 RUB,nomeal,83,8.5,8.7,https://ostrovok.ru/hotel/russia/moscow/mid10670964/north_star_hotel_9/?dates=01.03.2026-04.03.2...


In [10]:
import pandas as pd
from utils import ostrovok_url

# Compare scoring results with reference scores
if reference_scores and scoring_result and not scoring_result["error"]:
    # Sort reference hotels by evf_score (descending)
    sorted_ref = sorted(
        reference_scores.items(),
        key=lambda x: x[1]["evf_score"],
        reverse=True
    )

    # Create rank mapping: hid -> rank (1-indexed)
    ref_rank_map = {hid: idx + 1 for idx, (hid, _) in enumerate(sorted_ref)}

    # Build comparison table
    comparison_data = []
    for idx, result in enumerate(scoring_result["results"], 1):
        hotel_id = result["hotel_id"]
        llm_score = result["score"]

        # Find hotel in combined to get hid
        hotel = next((h for h in combined if h["id"] == hotel_id), None)
        if not hotel:
            continue

        hid = hotel["hid"]
        name = hotel["name"]

        # Get reference data
        ref_data = reference_scores.get(hid)
        if ref_data:
            evf_score = ref_data["evf_score"]
            ref_rank = ref_rank_map.get(hid, "N/A")
        else:
            evf_score = None
            ref_rank = "N/A"

        comparison_data.append({
            "llm_rank": idx,
            "name": name[:40],
            "hid": hid,
            "llm_score": llm_score,
            "evf_score": evf_score,
            "ref_rank": ref_rank,
            "rank_diff": idx - ref_rank if isinstance(ref_rank, int) else None,
        })

    # Display comparison
    comparison_df = pd.DataFrame(comparison_data)

    print(f"\n{'='*100}")
    print(f"COMPARISON: LLM SCORING vs REFERENCE")
    print(f"{'='*100}\n")
    print(comparison_df.to_string(index=False))

    # Calculate metrics
    valid_diffs = [d for d in comparison_df["rank_diff"] if d is not None]
    if valid_diffs:
        avg_diff = sum(abs(d) for d in valid_diffs) / len(valid_diffs)
        print(f"\n📊 Средняя разница рангов: {avg_diff:.1f}")
        print(f"   Отелей в сравнении: {len(valid_diffs)}/{len(comparison_data)}")

    # Show hotels not in reference with detailed info
    missing_in_ref = []
    for row in comparison_data:
        if row["evf_score"] is None:
            # Get full hotel data
            hotel = next((h for h in combined if h["hid"] == row["hid"]), None)
            if hotel:
                # Get LLM result details
                result = next((r for r in scoring_result["results"] if r["hotel_id"] == hotel["id"]), None)
                
                # Get reviews
                reviews = hotel.get("reviews", {})
                avg_rating = reviews.get("avg_rating") if reviews else None
                detailed_avg = reviews.get("detailed_averages", {}) if reviews else {}
                
                # Get selected rate price
                selected_hash = result.get("selected_rate_hash") if result else None
                rates = hotel.get("rates", [])
                selected_rate = next((r for r in rates if r.get("match_hash") == selected_hash), None)
                
                price_str = "N/A"
                if selected_rate:
                    daily_prices = selected_rate.get("daily_prices", [])
                    if daily_prices:
                        total_price = sum(float(p) for p in daily_prices)
                        pt = selected_rate.get("payment_options", {}).get("payment_types", [])
                        currency = pt[0].get("show_currency_code", "") if pt else ""
                        price_str = f"{total_price:.0f} {currency}"
                
                # Generate Ostrovok URL
                url = ostrovok_url(
                    hotel_id=hotel.get("id"),
                    hid=hotel.get("hid"),
                    checkin=CHECKIN_DATE,
                    checkout=CHECKOUT_DATE,
                    guests=GUESTS,
                    region_id=REGION_ID,
                )
                
                missing_in_ref.append({
                    "llm_rank": row["llm_rank"],
                    "name": hotel.get("name", ""),
                    "hid": row["hid"],
                    "llm_score": row["llm_score"],
                    "avg_rating": avg_rating,
                    "cleanness": detailed_avg.get("cleanness"),
                    "price": price_str,
                    "url": url,
                    "top_reasons": result.get("top_reasons", []) if result else [],
                })
        
elif not reference_scores:
    print("ℹ️  Референсные данные не загружены, сравнение невозможно")


COMPARISON: LLM SCORING vs REFERENCE

 llm_rank                                     name      hid  llm_score  evf_score  ref_rank  rank_diff
        1 Апартаменты MORE и SPA c сауной и видом  13319787         98      80.00         2         -1
        2 Квартира двухкомнатные Be Home на 25 окт  9992650         96      66.25        29        -27
        3 Апартаменты Изумруд Возле Аэропорта Засе 10002890         94      66.25        32        -29
        4    Квартира Двухкомнатная Студя в Центре 10095054         93      66.25        39        -35
        5            Апартаменты Красных Мадьяр 14  9769032         92      66.25        38        -33
        6 Апартаменты Колибри Премиум Класса в Цен 10530830         90      72.50        12         -6
        7         AZIMUT Отель Аэропорт Иркутск 4* 13317260         89      76.25         6          1
        8                            Отель Иркутск  7819551         85      61.25        84        -76
        9                      Оте